# Fulmine LABS Eyball
## Overview
This Python code implements a class wrapper around an Anomaly Detection model which can be used to visually check if an image is anomalous or not. The supported architecture for this model is 'Siamese Network'.
In order to perform reduce false negatives the code compares the image against a jury of randomly selected known good images of configurable size 'jury_size'. These images were studies obtained from the Cancer Image Database, stored in Orthanc and then modified using the Fulmine mini-PACS code.
If the number of jurors who vote that the image is simlar to the chosen known good image is below a configurable 'threshold' then the code returns a verdict of 'Anomalous', otherwise it returns a verdict of 'Normal'.
If an image path is not specified but screen coordinates are, these will be used instead, enabling direct integration with automated visual checking scripts.

One goal is to use this class as part of automating visual checking of a medical image (PACS) production pipeline, although it could theoretically visually check any type of image on which the model has been trained.

It also has the capability of describing the images, using GPT-4 Turbo Vision, if an OpenAI key is supplied in the 'Eyball-OpenAI_key.txt' file.

## Initialize the Eyball class

predictor = ModelPredictor(siamese_model_path, known_good_images_folder, Eyball_key, threshold, jury_size)

## Example calls

role = "You are a radiology PACS test engineer, analyzing PACS or test process related image anomalies"

image_description_directive = "If the image is obviously not a medical image, state *** ANOMALOUS ***. If it is a typical medical image as acquired by an imaging modality with no additions or enhancements, state *** NORMAL ***. Otherwise, if it is a medical image but it also clearly has textual overlays or annotations or digital or image processing artifacts that could have been added by the PACS image viewer technology, describe those features and append *** ANOMALOUS ***."

verdict = predictor.predict_siamese(test_image_path)

actual_description = predictor.describe_image(test_image_path, None, role, image_description_directive)

## Author
Duncan Henderson
Fulmine Labs LLC


In [1]:
import numpy as np
import os
import io
import cv2
from PIL import Image, ImageGrab
import logging
import random
import base64
import requests
from openai import OpenAI

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report



In [2]:
import os
import numpy as np
import cv2
from PIL import Image, ImageGrab
import base64
import io
import requests
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
import tensorflow as tf
from openai import OpenAI

class ModelPredictor:
    def __init__(self, siamese_model_path, known_good_images_folder, api_key_file='Eyball-OpenAI_key.txt', threshold=0.5, jury_size=12):
        self.siamese_model_path = siamese_model_path
        self.known_good_images_folder = known_good_images_folder
        self.api_key = self.load_api_key(api_key_file)
        self.client = OpenAI(api_key=self.api_key)
        self.siamese_model = self.load_siamese_model()
        self.threshold = threshold
        self.jury_size = jury_size
        self.known_good_images = self.preload_known_good_images()
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }

    def load_api_key(self, filename):
        try:
            with open(filename, 'r') as file:
                return file.read().strip()
        except FileNotFoundError:
            raise Exception(f"API key file not found: {filename}")

    def preload_known_good_images(self):
        print("Preloading known good images...")
        image_paths = []
        for root, dirs, files in os.walk(self.known_good_images_folder):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    full_path = os.path.join(root, file)
                    image_paths.append(full_path)
        return image_paths

    def load_siamese_model(self):
        input_shape = (152, 152, 1)
        input_a = Input(shape=input_shape)
        input_b = Input(shape=input_shape)
        base_network = self.initialize_base_network(input_shape)
        processed_a = base_network(input_a)
        processed_b = base_network(input_b)
        distance = Lambda(lambda tensors: tf.sqrt(tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1)))([processed_a, processed_b])
        model = Model(inputs=[input_a, input_b], outputs=distance)
        model.load_weights(self.siamese_model_path)
        print("Siamese model loaded successfully.")
        return model

    def initialize_base_network(self, input_shape):
        input = Input(shape=input_shape)
        x = Conv2D(64, (3, 3), activation='relu')(input)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(128, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        return Model(inputs=input, outputs=x)

    
    def preprocess_image(self, image=None, image_path=None):
        if image_path:
            # Load image from file path
            try:
                image = Image.open(image_path)
            except FileNotFoundError as e:
                print(f"Failed to open image at {image_path}: {e}")
                return None
            except Exception as e:
                print(f"Error processing image at {image_path}: {e}")
                return None
        if isinstance(image, np.ndarray):
            # Convert to PIL Image for consistent processing
            image = Image.fromarray(image.astype('uint8'))
    
        if image is None:
            print("No image provided for preprocessing")
            return None
    
        # Convert to grayscale and resize
        image = image.convert('L')  # Convert to grayscale
        target_size = (152, 152)
        image = image.resize(target_size)
        image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
        if image.ndim == 2:  # Ensure image has 3 dimensions if it's still 2D
            image = np.expand_dims(image, -1)
    
        return image

    def capture_screen(self, coordinates):
        screenshot = ImageGrab.grab(bbox=coordinates)
        return np.array(screenshot, dtype=np.uint8)  # Ensure dtype is uint8

    def predict_siamese(self, image_path=None, coordinates=None):
        if coordinates:
            print("Capturing screen...")
            captured_image = self.capture_screen(coordinates)
            if captured_image is None:
                print("Failed to capture screen")
                return None
            image = self.preprocess_image(image=captured_image)
        elif image_path:
            image = self.preprocess_image(image_path=image_path)
            if image is None:
                print("Failed to preprocess image from path")
                return None
        else:
            raise ValueError("Either image_path or coordinates must be provided.")

    def predict_siamese(self, image_path=None, coordinates=None):
        if coordinates:
            print("Capturing screen...")
            captured_image = self.capture_screen(coordinates)
            if captured_image is None:
                print("Failed to capture screen")
                return None
            image = self.preprocess_image(image=captured_image)
        elif image_path:
            image = self.preprocess_image(image_path=image_path)
        else:
            raise ValueError("Either image_path or coordinates must be provided.")
    
        if image is None:
            print("No image to process")
            return None
    
        image = np.expand_dims(image, axis=0)  # Adjust as necessary for the model input
    
        print("Image loaded and processed, predicting...")

        votes = []
        for known_good_image_path in random.sample(self.known_good_images, min(self.jury_size, len(self.known_good_images))):
            known_good_image = self.preprocess_image(image_path=known_good_image_path)
            if known_good_image is None:
                continue  # Skip if image can't be processed
            known_good_image = np.expand_dims(known_good_image, axis=0)

            # Prepare the pair
            image_pair = [image, known_good_image]

            # Make prediction
            prediction_distance = self.siamese_model.predict(image_pair)
            is_similar = prediction_distance < self.threshold  # Threshold to determine similarity
            print(f"Comparing {image_path if image_path else 'screen capture'} with {known_good_image_path}: Distance = {prediction_distance}, Similar = {is_similar}")
            votes.append(is_similar)

        # Calculate the majority vote
        num_similar = sum(votes)
        majority_similar = num_similar > len(votes) / 2
        print(f"Total votes for 'Similar': {num_similar}/{len(votes)}. Final verdict: {'Normal' if majority_similar else 'Anomalous'}")

        return 'Normal' if majority_similar else 'Anomalous'

    def describe_image(self, image_path=None, coordinates=None, role_description="User", image_description_directive="Describe the image"):
        try:
            if coordinates:
                print("Capturing screen for description...")
                image = self.capture_screen(coordinates)
            elif image_path:
                print("Loading image from path for description...")
                image = self.preprocess_image(image_path=image_path)
            else:
                raise ValueError("Either image_path or coordinates must be provided.")
    
            if image is None:
                raise ValueError("Failed to load or process image.")
    
            print("Encoding image for API request...")
            base64_image = self.encode_image(image)
            description = self.send_image_to_api(base64_image, role_description, image_description_directive)
            print("Description received.")
            return description
        except Exception as e:
            print(f"Error in describe_image: {str(e)}")
            return None

    def encode_image(self, image):
        """Converts a numpy array image to JPEG base64."""
        try:
            if image.ndim == 3 and image.shape[2] == 1:  # Check if it's single-channel
                image = image.squeeze(-1)  # Remove the last dimension if it's single-channel
            if isinstance(image, np.ndarray):
                # Ensure the data type is uint8
                image = (image * 255).clip(0, 255).astype(np.uint8)
                # Convert numpy array to PIL Image
                if image.ndim == 2:  # Grayscale
                    image = Image.fromarray(image, 'L')
                else:
                    image = Image.fromarray(image, 'RGB')
            buffer = io.BytesIO()
            image.save(buffer, format="JPEG")
            encoded_string = base64.b64encode(buffer.getvalue()).decode('utf-8')
            return encoded_string
        except Exception as e:
            raise ValueError(f"Error encoding image: {str(e)}")

    def send_image_to_api(self, base64_image, role_description, image_description_directive):
        print("Sending image to API...")
        payload = {
            #"model": "gpt-4-turbo",
            "model": "gpt-4o",
            "messages": [
                {
                    "role": "system",
                    "content": role_description
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": image_description_directive
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=self.headers, json=payload)
        if response.status_code != 200:
            print("Error from API:", response.status_code, response.text)
            return None

        try:
            description = response.json()['choices'][0]['message']['content']
            return description
        except KeyError as e:
            print("Failed to parse API response:", response.json())
            raise e



In [19]:
known_good_images_folder = r"d:\training_images\test\valid" # Generated by running the Fulmine mini-PACS archive code on Orthanc studies
siamese_model_path = r'models\lung_ct_siamese_network_weights_043024.h5' # Generated by running the Fulmine mini-PACS model generation code

api_key_file='Eyball-OpenAI_key.txt'

jury_size=12
threshold = 0.5

# LLM prompts
role = "You are a radiology PACS test engineer, analyzing PACS or test process related image anomalies"
image_description_directive = "If the image is obviously not a medical image, state *** ANOMALOUS ***. If it is a typical medical image as acquired by an imaging modality with no additions or enhancements, state *** NORMAL ***. Otherwise, if it is a medical image but it also clearly has textual overlays or annotations or digital or image processing artifacts that could have been added by the PACS image viewer technology, describe those features and append *** ANOMALOUS ***."


In [20]:
predictor = ModelPredictor(siamese_model_path, known_good_images_folder, api_key_file, threshold, jury_size)

Siamese model loaded successfully.
Preloading known good images...


In [21]:
# Capture and evaluate an area of the screen ...

left = 10
right = 500
top = 10
bottom = 200

siamese_result = predictor.predict_siamese(coordinates=(left, top, right, bottom))
print ("Siamese result", siamese_result)

predictor.describe_image(coordinates=(left, top, right, bottom), role_description=role, image_description_directive=image_description_directive)

Capturing screen...
Image loaded and processed, predicting...
1/1 [==============================] - 0s 115ms/step
Comparing screen capture with d:\training_images\test\valid\zoomed\randomized_wl\d3dd5f66-6abd-4615-8745-77f35d6fdfc8_1.png: Distance = [1.1453431], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing screen capture with d:\training_images\test\valid\dummy_class\e9406de0-2d76-44a3-91bd-195efd0badd6.png: Distance = [1.1453431], Similar = [False]
1/1 [==============================] - 0s 35ms/step
Comparing screen capture with d:\training_images\test\valid\zoomed\randomized_wl\4bc169b8-0378-46c8-9538-746a2b29b964_0.png: Distance = [1.1453431], Similar = [False]
1/1 [==============================] - 0s 29ms/step
Comparing screen capture with d:\training_images\test\valid\randomized_wl\cropped\fdfcea90-86a8-4337-96f3-5359b08ea808_0.png: Distance = [1.1453431], Similar = [False]
1/1 [==============================] - 0s 40ms/step
Comparing screen ca

'*** ANOMALOUS ***'

In [6]:
# Or (from here on) pass in a captured and saved file

test_image_path = r'C:\temp\engineer_typing3.png'
print("Model predicts", predictor.predict_siamese(image_path=test_image_path))

actual_description = predictor.describe_image(image_path=test_image_path, role_description=role, image_description_directive=image_description_directive)
print ("LLM description", actual_description)


Image loaded and processed, predicting...
1/1 [==============================] - 0s 17ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\zoomed\randomized_wl\0ac5863d-51b1-460f-abe1-3e330c596585_0.png: Distance = [0.93199706], Similar = [False]
1/1 [==============================] - 0s 16ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\zoomed\randomized_wl\169fb57b-cbec-45ba-b44e-468b05185734_0.png: Distance = [0.9382483], Similar = [False]
1/1 [==============================] - 0s 22ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\zoomed\randomized_wl\0b2a862a-5b50-4e41-a0cf-910364028702_1.png: Distance = [1.0572661], Similar = [False]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\engineer_typing3.png with d:\training_images\test\valid\dummy_class\9a550bc6-cd12-41a4-b231-c59e1e866a38.png: Distance = [0.714512], Similar = [False]
1/1 [==============================] 

In [7]:
test_image_path = r'Custom_invalid\cat.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 47ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\dummy_class\b171c664-0dcc-403e-aa6b-5f22a804d79d.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\randomized_wl\cropped\9c35c6da-c3ce-40d6-9b5c-b8757939def7_0.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 52ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\dummy_class\aa772b65-80ea-47ea-9971-7b114a33d5ee.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing Custom_invalid\cat.jpg with d:\training_images\test\valid\randomized_wl\cropped\b2744d7c-aa1a-4c33-9bb2-43203897b436_1.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 33ms/step
Comparing Custom_inval

In [8]:
test_image_path = r'custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\training_images\test\valid\randomized_wl\cropped\92c3b500-973b-41d3-9149-9e803d33cf0a_0.png: Distance = [0.3196137], Similar = [ True]
1/1 [==============================] - 0s 44ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\training_images\test\valid\zoomed\randomized_wl\c0a276e3-ac4a-48c3-b623-f5c8784caa78_1.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 53ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\training_images\test\valid\randomized_wl\cropped\1869024e-51c7-4b19-a1c5-698731f1ddb2_0.png: Distance = [0.02389454], Similar = [ True]
1/1 [==============================] - 0s 36ms/step
Comparing custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with d:\

In [9]:
test_image_path = r'custom_test_valid\istockphoto-493741910-612x612.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 51ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\dummy_class\f5c14d60-f5b2-4254-acd6-aef2c93cfb32.png: Distance = [0.14590386], Similar = [ True]
1/1 [==============================] - 0s 50ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\zoomed\randomized_wl\68ca8240-9d01-490a-a83c-6d8a4041c598_1.png: Distance = [0.14590386], Similar = [ True]
1/1 [==============================] - 0s 35ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\randomized_wl\cropped\52a6abd1-5252-47a1-bc4e-e43ee9e2bc98_1.png: Distance = [0.18324384], Similar = [ True]
1/1 [==============================] - 0s 34ms/step
Comparing custom_test_valid\istockphoto-493741910-612x612.jpg with d:\training_images\test\valid\randomized_wl\cropped\e6e337d4-f0cb-460f-97d0-b6

In [10]:
test_image_path = r'custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 49ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with d:\training_images\test\valid\randomized_wl\cropped\ef1c684d-a6e6-4151-ba12-66109e84d2b9_0.png: Distance = [0.6845648], Similar = [False]
1/1 [==============================] - 0s 30ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with d:\training_images\test\valid\dummy_class\7e11f3b3-2e37-4fdf-b9f3-b4f4c8cb60e5.png: Distance = [0.71923923], Similar = [False]
1/1 [==============================] - 0s 34ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with d:\training_images\test\valid\dummy_class\7b786536-f33f-4f61-b29a-f9f8aad246ea.png: Distance = [0.30020693], Similar = [ True]
1/1 [==============================] - 0s 31ms/step
Comparing custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with d:\training_im

In [11]:
test_image_path = r'custom_invalid\istockphoto-with_arrow.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 33ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\zoomed\randomized_wl\6b6ad9fa-d97b-4a0c-9e43-b65cf5c190d4_1.png: Distance = [0.8484544], Similar = [False]
1/1 [==============================] - 0s 46ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\zoomed\randomized_wl\21e8436d-428c-4094-abcc-ffee904db7b5_1.png: Distance = [0.8484544], Similar = [False]
1/1 [==============================] - 0s 37ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\randomized_wl\cropped\f0a1e6ca-66a9-4356-8745-393d7cd1bfd2_1.png: Distance = [0.8484544], Similar = [False]
1/1 [==============================] - 0s 52ms/step
Comparing custom_invalid\istockphoto-with_arrow.jpg with d:\training_images\test\valid\randomized_wl\cropped\c0eb3d34-b0ce-4714-a4f0-15a529bcb002_0.png: Distance = [0.

In [12]:
test_image_path = r'custom_invalid\Lung_abscess_-_CT_with_overlay.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 27ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\randomized_wl\cropped\42f43890-0827-41b0-8f7f-4b93d9b6c7c4_0.png: Distance = [0.88091654], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\randomized_wl\cropped\a0729163-7784-4ed2-b8cd-e7b280c55ff5_0.png: Distance = [1.0645854], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\zoomed\randomized_wl\965bfc9b-0a90-40ae-bc78-9644b3866718_0.png: Distance = [1.1311321], Similar = [False]
1/1 [==============================] - 0s 40ms/step
Comparing custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with d:\training_images\test\valid\zoomed\randomized_wl\03173cc0-7bb3-4125-9ee9-9

In [13]:
test_image_path = r'Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 47ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\valid\randomized_wl\cropped\b8467724-8339-4a5c-9b90-0810ad0183a9_1.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\valid\zoomed\randomized_wl\45807559-6251-47ad-87c8-424d54243981_1.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 27ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\valid\dummy_class\7a2e1cd7-2454-4b00-8347-6e900a89b9db.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 34ms/step
Comparing Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with d:\training_images\test\v

In [14]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 34ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with d:\training_images\test\valid\randomized_wl\cropped\c33d8b9e-af3d-4fe7-801a-3a3dd92f48d1_1.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 61ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with d:\training_images\test\valid\randomized_wl\cropped\2dc9cf4f-a965-4499-b458-b60c866f3077_0.png: Distance = [0.7176508], Similar = [False]
1/1 [==============================] - 0s 26ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with d:\training_images\test\valid\zoomed\randomized_wl\f41cb11b-fbfe-4a1d-9b9d-eb618ac769b3_1.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 32ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modi

In [15]:
test_image_path = r'Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))

Image loaded and processed, predicting...
1/1 [==============================] - 0s 38ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with d:\training_images\test\valid\zoomed\randomized_wl\4c182471-ca37-4264-869e-61aec20a7f81_0.png: Distance = [0.77528584], Similar = [False]
1/1 [==============================] - 0s 46ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with d:\training_images\test\valid\zoomed\randomized_wl\e0ba9897-5cb9-4b21-8f9d-df15ac352b13_1.png: Distance = [0.77528584], Similar = [False]
1/1 [==============================] - 0s 45ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with d:\training_images\test\valid\zoomed\randomized_wl\d12a4d18-8ea6-47e7-9007-88c2200fc50c_0.png: Distance = [0.77528584], Similar = [False]
1/1 [==============================] - 0s 47ms/step
Comparing Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.j

In [16]:
test_image_path = r'Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 40ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\dummy_class\ad82138d-1137-4831-a176-24a5df99d727.png: Distance = [0.45649433], Similar = [ True]
1/1 [==============================] - 0s 53ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\randomized_wl\cropped\42f43890-0827-41b0-8f7f-4b93d9b6c7c4_1.png: Distance = [0.00941378], Similar = [ True]
1/1 [==============================] - 0s 42ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\dummy_class\ffbdd0f2-8cea-4eae-9652-511bc7406689.png: Distance = [0.45649433], Similar = [ True]
1/1 [==============================] - 0s 47ms/step
Comparing Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with d:\training_images\test\valid\zoomed\randomized_w

In [17]:
def evaluate_methods_simplified(base_folder, sample_size=40, jury_size=12, role="User", image_description_directive="Describe the image"):
    valid_folder = os.path.join(base_folder, 'valid')
    invalid_folder = os.path.join(base_folder, 'invalid')

    # Ensure directories exist
    if not os.path.exists(valid_folder) or not os.path.exists(invalid_folder):
        raise ValueError("One or more image directories do not exist.")

    valid_images = random.sample(os.listdir(valid_folder), min(sample_size, len(os.listdir(valid_folder))))
    invalid_images = random.sample(os.listdir(invalid_folder), min(sample_size, len(os.listdir(invalid_folder))))
    
    predictions_siamese = []
    predictions_gpt = []
    actuals = [1] * len(valid_images) + [0] * len(invalid_images)

    for filename in valid_images + invalid_images:
        folder = valid_folder if filename in valid_images else invalid_folder
        image_path = os.path.join(folder, filename)

        siamese_result = predictor.predict_siamese(image_path)
        print ("Siamese prediction", siamese_result)
        predictions_siamese.append(1 if siamese_result == 'Normal' else 0)

        description_result = predictor.describe_image(image_path, None, role, image_description_directive)
        print ("LLM prediction", description_result)
        predictions_gpt.append(1 if 'NORMAL' in description_result.upper() else 0)

    accuracy_s = accuracy_score(actuals, predictions_siamese)
    precision_s = precision_score(actuals, predictions_siamese)
    recall_s = recall_score(actuals, predictions_siamese)
    f1_s = f1_score(actuals, predictions_siamese)
    
    accuracy_g = accuracy_score(actuals, predictions_gpt)
    precision_g = precision_score(actuals, predictions_gpt)
    recall_g = recall_score(actuals, predictions_gpt)
    f1_g = f1_score(actuals, predictions_gpt)

    print('Evaluation Results - Siamese Model:', {'accuracy': accuracy_s, 'precision': precision_s, 'recall': recall_s, 'f1': f1_s})
    print('Evaluation Results - GPT Model:', {'accuracy': accuracy_g, 'precision': precision_g, 'recall': recall_g, 'f1': f1_g})

    return {
        "siamese": {"accuracy": accuracy_s, "precision": precision_s, "recall": recall_s, "f1": f1_s},
        "gpt": {"accuracy": accuracy_g, "precision": precision_g, "recall": recall_g, "f1": f1_g}
    }



In [23]:
# Example call
base_folder = r'model_comparison_test'
sample_size = 40
try:
    results = evaluate_methods_simplified(base_folder, sample_size, jury_size, role, image_description_directive)
    print("Evaluation Results:", results)
except Exception as e:
    print("Error during evaluation:", str(e))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 48ms/step
Comparing model_comparison_test\valid\1faacb99-86d9-4ac5-80b0-40c41d1f1a6d.png with d:\training_images\test\valid\randomized_wl\cropped\a376098e-2e5a-402c-bc11-ff8b918b272c_1.png: Distance = [0.15160172], Similar = [ True]
1/1 [==============================] - 0s 43ms/step
Comparing model_comparison_test\valid\1faacb99-86d9-4ac5-80b0-40c41d1f1a6d.png with d:\training_images\test\valid\zoomed\randomized_wl\ef55f33f-5d45-4342-a5b9-89baf552a556_0.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 47ms/step
Comparing model_comparison_test\valid\1faacb99-86d9-4ac5-80b0-40c41d1f1a6d.png with d:\training_images\test\valid\randomized_wl\cropped\4d70ad68-deef-4986-9dd3-5931690173eb_1.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 52ms/step
Comparing model_comparison_test\valid\1faacb99-86d9-4ac5-80b0-40c41d1f1a6d.png with d:\training_im